In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
import re
import unicodedata
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import time

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 상수 정의
SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence
UNK_token = 2  # Unknown token
MAX_LENGTH = 15
teacher_forcing_ratio = 0.5

class Lang:
    """언어별 단어 사전을 관리하는 클래스"""
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
        self.n_words = 3  # SOS, EOS, UNK 토큰 포함

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            if word.strip():  # 빈 문자열 제외
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def normalizeString(s):
    """문자열 정규화 - 개선된 버전"""
    # 소문자로 변환
    s = s.lower().strip()
    # 특수문자 처리
    s = re.sub(r"([.!?])", r" \1", s)
    # 불필요한 문자 제거 (한글, 영문, 숫자, 기본 구두점만 유지)
    s = re.sub(r"[^a-zA-Z가-힣0-9.!?]+", r" ", s)
    # 다중 공백을 단일 공백으로
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def readLangs(lang1, lang2, pairs):
    """언어 쌍을 읽고 처리"""
    print(f"Reading lines from {lang1} to {lang2}...")

    # 언어 객체 생성
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    # 정규화된 문장 쌍 생성
    normalized_pairs = []
    for pair in pairs:
        if len(pair) >= 2:  # 쌍이 올바른지 확인
            normalized_pair = [normalizeString(pair[0]), normalizeString(pair[1])]
            if normalized_pair[0] and normalized_pair[1]:  # 빈 문자열이 아닌지 확인
                normalized_pairs.append(normalized_pair)

    print(f"Read {len(normalized_pairs)} sentence pairs")
    return input_lang, output_lang, normalized_pairs

def filterPair(p):
    """문장 쌍 필터링 (길이 제한)"""
    return len(p[0].split(' ')) < MAX_LENGTH and \
           len(p[1].split(' ')) < MAX_LENGTH and \
           len(p[0].split(' ')) > 0 and \
           len(p[1].split(' ')) > 0

def filterPairs(pairs):
    """모든 문장 쌍 필터링"""
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, pairs):
    """데이터 준비"""
    input_lang, output_lang, pairs = readLangs(lang1, lang2, pairs)
    print(f"Read {len(pairs)} sentence pairs")

    pairs = filterPairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")

    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

# 확장된 한영 번역 데이터
sample_pairs = [
    # 기본 인사
    ["안녕하세요", "hello"],
    ["안녕", "hi"],
    ["안녕히 가세요", "goodbye"],
    ["잘 가요", "bye"],

    # 감사 및 사과
    ["감사합니다", "thank you"],
    ["고맙습니다", "thanks"],
    ["죄송합니다", "sorry"],
    ["미안해요", "sorry"],
    ["괜찮습니다", "it is okay"],
    ["괜찮아요", "it is fine"],

    # 일상 대화
    ["오늘 날씨가 좋네요", "the weather is nice today"],
    ["날씨가 나쁘네요", "the weather is bad"],
    ["밥 먹었어요", "i ate"],
    ["물 주세요", "give me water"],
    ["도움이 필요해요", "i need help"],
    ["도와주세요", "please help me"],

    # 질문
    ["어디에 있어요", "where is it"],
    ["몇 시예요", "what time is it"],
    ["이름이 뭐예요", "what is your name"],
    ["어떻게 지내세요", "how are you"],
    ["뭐 해요", "what are you doing"],

    # 학습 관련
    ["한국어를 공부해요", "i study korean"],
    ["영어를 배워요", "i learn english"],
    ["책을 읽어요", "i read books"],
    ["공부해요", "i study"],

    # 감정 표현
    ["좋아요", "i like it"],
    ["싫어요", "i do not like it"],
    ["행복해요", "i am happy"],
    ["슬퍼요", "i am sad"],
    ["피곤해요", "i am tired"],
    ["배고파요", "i am hungry"],

    # 시간 관련
    ["좋은 하루 되세요", "have a good day"],
    ["내일 봐요", "see you tomorrow"],
    ["잘 자요", "good night"],
    ["좋은 아침", "good morning"],

    # 기타
    ["사랑해요", "i love you"],
    ["맛있어요", "it is delicious"],
    ["예쁘네요", "it is pretty"],
    ["재미있어요", "it is fun"],
    ["어려워요", "it is difficult"],
    ["쉬워요", "it is easy"],

    # 더 많은 기본 표현
    ["네", "yes"],
    ["아니요", "no"],
    ["모르겠어요", "i do not know"],
    ["알겠어요", "i understand"],
    ["천천히", "slowly"],
    ["빨리", "quickly"],
    ["많이", "a lot"],
    ["조금", "a little"],

    # 가족
    ["엄마", "mother"],
    ["아빠", "father"],
    ["형", "brother"],
    ["누나", "sister"],

    # 음식
    ["물", "water"],
    ["밥", "rice"],
    ["빵", "bread"],
    ["커피", "coffee"],

    # 색깔
    ["빨간색", "red"],
    ["파란색", "blue"],
    ["하얀색", "white"],
    ["검은색", "black"]
]

# 데이터 준비
input_lang, output_lang, pairs = prepareData('korean', 'english', sample_pairs)
print("Sample pair:", random.choice(pairs))

class EncoderRNN(nn.Module):
    """단방향 인코더 RNN - 안정적인 버전"""
    def __init__(self, input_size, hidden_size, num_layers=2, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        # 단방향 GRU로 변경하여 크기 문제 해결
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, dropout=dropout, bidirectional=False)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    """개선된 어텐션 디코더 RNN - 단방향 인코더용"""
    def __init__(self, hidden_size, output_size, num_layers=2, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # 단방향 인코더이므로 크기 조정
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, num_layers, dropout=dropout_p)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        # 어텐션 계산 - hidden의 마지막 레이어만 사용
        last_hidden = hidden[-1].unsqueeze(0)  # 마지막 레이어의 hidden state
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], last_hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        # embedded + context를 결합
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    """문장을 인덱스 시퀀스로 변환 - UNK 토큰 처리 포함"""
    indexes = []
    for word in sentence.split(' '):
        if word in lang.word2index:
            indexes.append(lang.word2index[word])
        else:
            indexes.append(UNK_token)  # 알려지지 않은 단어
    return indexes

def tensorFromSentence(lang, sentence):
    """문장을 텐서로 변환"""
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    """문장 쌍을 텐서 쌍으로 변환"""
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """개선된 훈련 함수"""
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # 단방향이므로 hidden_size만 사용
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    # encoder_hidden이 이미 올바른 크기
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    # 그라디언트 클리핑
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), 1.0)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), 1.0)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.001):
    """개선된 반복 훈련 함수"""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        # 랜덤하게 훈련 쌍 선택
        training_pair = tensorsFromPair(random.choice(pairs))
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            elapsed = time.time() - start
            print(f'{iter} ({iter / n_iters * 100:.1f}%) {print_loss_avg:.4f} - {elapsed:.0f}s')

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """개선된 문장 번역 평가"""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        # 단방향이므로 hidden_size만 사용
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        # encoder_hidden이 이미 올바른 크기
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                if topi.item() in output_lang.index2word:
                    word = output_lang.index2word[topi.item()]
                    if word != 'UNK':  # UNK 토큰 제외
                        decoded_words.append(word)
                else:
                    decoded_words.append('<UNK>')

            decoder_input = topi.squeeze().detach()

        return decoded_words

def evaluateRandomly(encoder, decoder, n=5):
    """랜덤 문장들 평가"""
    print("\n=== 랜덤 테스트 결과 ===")
    for i in range(n):
        pair = random.choice(pairs)
        print(f'입력: {pair[0]}')
        print(f'정답: {pair[1]}')
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join([word for word in output_words if word != '<EOS>'])
        print(f'번역: {output_sentence}')
        print('-' * 30)

# 번역 챗봇 클래스
class TranslationChatbot:
    def __init__(self, encoder, decoder, input_lang, output_lang):
        self.encoder = encoder
        self.decoder = decoder
        self.input_lang = input_lang
        self.output_lang = output_lang

    def translate(self, sentence):
        """문장 번역"""
        try:
            normalized_sentence = normalizeString(sentence)
            if not normalized_sentence:
                return "빈 문장입니다."

            output_words = evaluate(self.encoder, self.decoder, normalized_sentence)
            translation = ' '.join([word for word in output_words if word not in ['<EOS>', '<UNK>']])

            if not translation:
                return "번역할 수 없습니다."

            return translation
        except Exception as e:
            return f"번역 오류: {str(e)}"

    def chat(self):
        """챗봇 대화 시작"""
        print("한영 번역 챗봇입니다! 한국어를 입력하면 영어로 번역해드립니다.")
        print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
        print("-" * 50)

        while True:
            user_input = input("한국어: ").strip()

            if user_input.lower() in ['quit', 'exit', '종료']:
                print("챗봇을 종료합니다. 안녕히 가세요!")
                break

            if not user_input:
                print("문장을 입력해주세요.")
                continue

            translation = self.translate(user_input)
            print(f"영어: {translation}")
            print("-" * 30)

# 개선된 모델 생성 및 훈련
print("모델을 생성하고 훈련을 시작합니다...")

hidden_size = 128
num_layers = 2

encoder = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers).to(device)

# 더 많은 반복으로 훈련
print("모델 훈련 중...")
trainIters(encoder, decoder, 15000, print_every=3000, learning_rate=0.001)

# 훈련 결과 평가
evaluateRandomly(encoder, decoder)

# 챗봇 생성
chatbot = TranslationChatbot(encoder, decoder, input_lang, output_lang)

# 예시 번역 테스트
print("\n=== 예시 번역 ===")
test_sentences = ["안녕하세요", "감사합니다", "좋은 하루 되세요", "배고파요", "행복해요"]
for sentence in test_sentences:
    translation = chatbot.translate(sentence)
    print(f"한국어: {sentence} -> 영어: {translation}")

print("\n모델 훈련이 완료되었습니다!")
print("대화형 챗봇을 사용하려면 다음 줄의 주석을 해제하세요:")
print("# chatbot.chat()")



Using device: cuda
Reading lines from korean to english...
Read 61 sentence pairs
Read 61 sentence pairs
Trimmed to 61 sentence pairs
Counting words...
Counted words:
korean 79
english 83
Sample pair: ['좋은 아침', 'good morning']
모델을 생성하고 훈련을 시작합니다...
모델 훈련 중...
3000 (20.0%) 0.4592 - 38s
6000 (40.0%) 0.0003 - 76s
9000 (60.0%) 0.0001 - 114s
12000 (80.0%) 0.0000 - 151s
15000 (100.0%) 0.0000 - 189s

=== 랜덤 테스트 결과 ===
입력: 괜찮아요
정답: it is fine
번역: it is fine
------------------------------
입력: 죄송합니다
정답: sorry
번역: sorry
------------------------------
입력: 예쁘네요
정답: it is pretty
번역: it is pretty
------------------------------
입력: 한국어를 공부해요
정답: i study korean
번역: i study korean
------------------------------
입력: 하얀색
정답: white
번역: white
------------------------------

=== 예시 번역 ===
한국어: 안녕하세요 -> 영어: hello
한국어: 감사합니다 -> 영어: thank you
한국어: 좋은 하루 되세요 -> 영어: have a good day
한국어: 배고파요 -> 영어: i am hungry
한국어: 행복해요 -> 영어: i am happy

모델 훈련이 완료되었습니다!
대화형 챗봇을 사용하려면 다음 줄의 주석을 해제하세요:
# chatbot.chat()


In [9]:
# 대화형 챗봇 시작 (주석 해제하여 사용)
chatbot.chat()

한영 번역 챗봇입니다! 한국어를 입력하면 영어로 번역해드립니다.
종료하려면 'quit' 또는 'exit'를 입력하세요.
--------------------------------------------------
한국어: 감사합니다.
영어: thank you
------------------------------
한국어: 배고파요
영어: i am hungry
------------------------------
한국어: 좋아요
영어: i like it
------------------------------
한국어: 좋은 아침
영어: good morning
------------------------------


KeyboardInterrupt: Interrupted by user